In [1]:
import os 
from pathlib import Path
abspath = Path('C:\\Users\\harkum\\Documents\\6311\\Software')
os.chdir(abspath)

<h2 style=' color: 	#0bff00;border-bottom: 2px dotted #ff0000;'> Copy Register data and Paste </h2>

```Copy__register address__register address``` <i> example :~ Copy__0xCB[7:4]__0xCC[3:0] </i>

<h3 style='color:#ffac00'> Parse all Copy Register data and Paste it to another Register </h3>

In [5]:
from typing import List
import json
import random
import re 
import pandas as pd 
copy_instructions = {}
i=0
for sh in ['Digital','Reference','Boost','DAC+PA','AZ_COMP']:
    sheet = 'raw_data/'+sh+'_data.csv'
    data = pd.read_csv(sheet)
    for col in data.columns.to_list()[1:]:
        instructions = data[col][1].split('\n')
        for instruction in instructions:
            if re.match('copy', instruction.lower()):
                copy_instructions.update({i : {
                    'page':sh,
                    'dft':col,
                    'ins':instruction
                }})
                i+=1
print(copy_instructions)
with open('raw_data/json/copy_parsing_raw_dump.json', 'w') as file:
    json.dump(copy_instructions, file)

{0: {'page': 'DAC+PA', 'dft': 'DAC-PA OFFSET CALIBRATION DIGITAL_IN (both PDM, TDM)\nThis procedure has the aim to measure the offset when the part will be in idle mode\n', 'ins': 'Copy__0xCB[7:4]__0xCC[3:0] "int/ext zero copied in int/ext idle"'}, 1: {'page': 'DAC+PA', 'dft': 'DAC-PA OFFSET CALIBRATION DIGITAL_IN (both PDM, TDM)\nThis procedure has the aim to measure the offset when the part will be in idle mode\n', 'ins': 'Copy__0xCA[7:4]__0xCB[3:0] "int/ext zero copied in int/ext idle"'}, 2: {'page': 'DAC+PA', 'dft': 'DAC-PA OFFSET CALIBRATION DIGITAL_IN (both PDM, TDM)\nThis procedure has the aim to measure the offset when the part will be in idle mode\n', 'ins': 'Copy__0xCA[3:0]__0xCC[7:4] "int/ext zero copied in int/ext idle"'}}


<h3 style = 'color:#00f6ff'> Extract the Register Address from copy Command</h3>

In [52]:
# This is the Approch 
pattern1=re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]")
re.findall(pattern1, '0xCB[7:4]')
'Copy__0xCB[7:4]__0xCC[3:0]'.split('__')[1:]

['0xCB[7:4]', '0xCC[3:0]']

In [58]:
def extract_RegisterAddress__Instruction(instruction: str)->dict:
    pattern1=re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]")
    register ={}
    def register_format(instrunction):
        instruction_length = len(instrunction)
        register = {}
        # there is no bit field it is the default size
        if instruction_length == 2:
            if ":" in instrunction[1]:
                bits = instrunction[1].split(':')
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : None,
                }
            else:
                register={
                    "RegAddr" : hex(int(instrunction[0],16)),
                    "MSB" : 7,
                    "LSB" : 0,
                    "Data" : hex(int(instrunction[1],16)),
                }
        # check for the bit field 
        elif instruction_length == 3:
            #check for the single bit or bit field 
            if ":" in instrunction[1]:
                bits = instrunction[1].split(':')
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : hex(int(instrunction[2],16)),
                }

            else:
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(instrunction[1],16)),
                "LSB" : hex(int(instrunction[1],16)),
                "Data" : hex(int(instrunction[2],16)),
                }
        return register
    
    if 'copy__' in instruction.lower():
        if len(regs := [re.findall(pattern1, i)[0] for i in instruction.split('__')[1:] if re.match(pattern1, i)]) == 2:
            if copyReg := register_format(regs[0]):
                if pasteReg := register_format(regs[1]):
                    register={
                        "copyReg":copyReg,
                        "pasteReg":pasteReg,
                    }
        else:
            register =register
    else:
        register = register
    
    return register
extract_RegisterAddress__Instruction('Copy__0xCB[7:4]__0xCC[3:0]')

{'copyReg': {'RegAddr': '0xcb', 'MSB': '0x4', 'LSB': '0x7', 'Data': None},
 'pasteReg': {'RegAddr': '0xcc', 'MSB': '0x0', 'LSB': '0x3', 'Data': None}}

<h3 style='color: #fff100'> Test Copy Instruction </h3>

In [59]:
clean_copy_instructions = {}
for i,instruction_set in copy_instructions.items():
    # print(instruction_set)
    instruction = instruction_set.get('ins')
    if register_instruction_parse := extract_RegisterAddress__Instruction(instruction):
        # print(register_instruction_parse)
        clean_copy_instructions.update({
            i:{
                'set':instruction_set,
                'clean':register_instruction_parse
            }
        })
    else:
        print(instruction)
        clean_copy_instructions.update({
            i:None
        })
with open('raw_data/json/clean_copy_instructions.json', 'w') as file:
    json.dump(clean_copy_instructions, file)